In [1]:
import pandas as pd

This notebook,
> 1. prepares LTE adoptation data in both edgelist and long format

> Steps :
    * LTE.xlsx file does not have country codes. This notebook meregd one WDI and LTE datasets to retains ISO_ALPHA codes.
    * Reads `nodal_time_series.csv` file and retains the same 74 countries in the LTE data
    * Stacks the year columns and makes the LTE data long format

> 2. prepares country level attribute datasets

> Steps:
    * Merges the node level time series data with the LTE long format one and then splits it on the TIME_PERIOD column to make 21 attribute dataframes
    * Merges country level fixed attribute data with the all of the attributeYEAR dataframes   

In [2]:
wdi = pd.read_csv("../data/raw/702e5907-a97c-4e0a-9f8f-511ea9b80ab0_Data.csv")

In [3]:
wdi = wdi.rename(columns = {'Country Name': 'country'})
wdi = wdi.rename(columns = {'Country Code': 'iso_o'})

In [4]:
wdi = wdi[['country', 'iso_o']]

In [42]:
wdi.head(2)

,country,iso_o
0,Afghanistan,AFG
1,Albania,ALB


In [6]:
lte = pd.read_excel("../data/raw/LTE.xlsx")

In [7]:
lte = lte.fillna(0)

In [8]:
lte.columns

Index(['country',      2000,      2001,      2002,      2003,      2004,
            2005,      2006,      2007,      2008,      2009,      2010,
            2011,      2012,      2013,      2014,      2015,      2016,
            2017,      2018,      2019,      2020],
      dtype='object')

In [9]:
lte_iso = wdi.merge(lte, on = 'country', how = 'left')

In [10]:
#lte_iso.tail(10)

In [11]:
lte_iso[lte_iso['iso_o'] == 'IND']

,country,iso_o,2000,2001,2002,2003,2004,2005,2006,2007,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
89,India,IND,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [12]:
lte_iso = lte_iso.iloc[:265, :]

In [40]:
missing1 = lte_iso.isna().sum()
#missing1

In [14]:
time_series = pd.read_csv("../data/cleaned/nodal_time_series.csv")

In [15]:
countries = list(set(time_series['iso_o'].unique()))

In [16]:
len(countries)

74

In [17]:
lte_iso = lte_iso[lte_iso['iso_o'].isin(countries)]

In [18]:
lte_iso['iso_o'].nunique()

74

In [32]:
lte_iso = lte_iso.fillna(0)

In [33]:
lte_iso_long = lte_iso.set_index(['iso_o', 'country']).stack().reset_index()

In [51]:
lte_iso_long.head(2)

,iso_o,country,TIME_PERIOD,LTE
0,ARG,Argentina,2000,0.0
1,ARG,Argentina,2001,0.0


In [35]:
lte_iso_long = lte_iso_long.rename(columns = {'level_2': 'TIME_PERIOD'})
lte_iso_long = lte_iso_long.rename(columns = {0: 'LTE'})

In [36]:
print(lte_iso['iso_o'].nunique())
print(lte_iso_long['iso_o'].nunique())
print(lte_iso_long['TIME_PERIOD'].nunique())

74
74
21


In [39]:
# sanity check
#lte_iso[lte_iso['iso_o'] == 'CIV']

In [49]:
time_series.head(2)

,iso_o,time_period,internet,gdp_pct,schooling,log_gdp_pct,country_id,D_log_gdp_pct,D_internet
1,ARG,2000,7.04,10631.65,15.68757,9.271591,ARG,-0.019252,3.76
2,ARG,2001,9.78,10051.94,16.30578,9.215521,ARG,-0.056070,2.74


In [48]:
# removing 1999 from time_series dataframe
time_series = time_series[time_series['time_period'] != 1999]

In [50]:
time_series= time_series.rename(columns = {'time_period': 'TIME_PERIOD'})
time_series= time_series.drop(columns = 'country_id')

In [53]:
merged1 = lte_iso_long.merge(time_series, on = ['iso_o', 'TIME_PERIOD'], how = 'left')

In [56]:
merged1.head(3)

,iso_o,country,TIME_PERIOD,LTE,internet,gdp_pct,schooling,log_gdp_pct,D_log_gdp_pct,D_internet
0,ARG,Argentina,2000,0.0,7.04,10631.6500,15.687570,9.271591,-0.019252,3.76
1,ARG,Argentina,2001,0.0,9.78,10051.9400,16.305780,9.215521,-0.056070,2.74
2,ARG,Argentina,2002,0.0,10.90,8861.5596,16.392509,9.089479,-0.126042,1.12


In [57]:
merged1.to_csv("../data/cleaned/nodal_time_series.csv", encoding = 'utf-8', index = False)
lte_iso.to_csv("../data/cleaned/LTE.csv", encoding = 'utf-8', index = False)
lte_iso_long.to_csv("../data/cleaned/LTE_long.csv", encoding = 'utf-8', index = False)

#### Country level attribute data

In [58]:
merged1 = merged1.sort_values('TIME_PERIOD')

In [60]:
merged1[(merged1['TIME_PERIOD'] == 2013) & (merged1['iso_o'] == 'BGD')]

,iso_o,country,TIME_PERIOD,LTE,internet,gdp_pct,schooling,log_gdp_pct,D_log_gdp_pct,D_internet
76,BGD,Bangladesh,2013,1.0,6.63,1103.16,10.348932,7.005934,0.049017,1.63


In [62]:
attribute2000 = merged1[merged1['TIME_PERIOD'] == 2000].sort_values('iso_o').reset_index(drop = True)

In [63]:
# sanity check
attribute2000[attribute2000['iso_o'] == 'BGD']

,iso_o,country,TIME_PERIOD,LTE,internet,gdp_pct,schooling,log_gdp_pct,D_log_gdp_pct,D_internet
4,BGD,Bangladesh,2000,0.0,0.07,620.56,7.566327,6.430622,0.033309,0.03


In [64]:
attribute2000['iso_o'].nunique()

74

In [ ]:
# sanity check
time_series[(time_series['TIME_PERIOD'] == 2000) & (time_series['iso_o'] == 'BGD')]

In [65]:
attribute2001 = merged1[merged1['TIME_PERIOD'] == 2001].sort_values('iso_o').reset_index(drop = True)
attribute2002 = merged1[merged1['TIME_PERIOD'] == 2002].sort_values('iso_o').reset_index(drop = True)
attribute2003 = merged1[merged1['TIME_PERIOD'] == 2003].sort_values('iso_o').reset_index(drop = True)
attribute2004 = merged1[merged1['TIME_PERIOD'] == 2004].sort_values('iso_o').reset_index(drop = True)
attribute2005 = merged1[merged1['TIME_PERIOD'] == 2005].sort_values('iso_o').reset_index(drop = True)
attribute2006 = merged1[merged1['TIME_PERIOD'] == 2006].sort_values('iso_o').reset_index(drop = True)
attribute2007 = merged1[merged1['TIME_PERIOD'] == 2007].sort_values('iso_o').reset_index(drop = True)
attribute2008 = merged1[merged1['TIME_PERIOD'] == 2008].sort_values('iso_o').reset_index(drop = True)
attribute2009 = merged1[merged1['TIME_PERIOD'] == 2009].sort_values('iso_o').reset_index(drop = True)
attribute2010 = merged1[merged1['TIME_PERIOD'] == 2010].sort_values('iso_o').reset_index(drop = True)
attribute2011 = merged1[merged1['TIME_PERIOD'] == 2011].sort_values('iso_o').reset_index(drop = True)
attribute2012 = merged1[merged1['TIME_PERIOD'] == 2012].sort_values('iso_o').reset_index(drop = True)
attribute2013 = merged1[merged1['TIME_PERIOD'] == 2013].sort_values('iso_o').reset_index(drop = True)
attribute2014 = merged1[merged1['TIME_PERIOD'] == 2014].sort_values('iso_o').reset_index(drop = True)
attribute2015 = merged1[merged1['TIME_PERIOD'] == 2015].sort_values('iso_o').reset_index(drop = True)
attribute2016 = merged1[merged1['TIME_PERIOD'] == 2016].sort_values('iso_o').reset_index(drop = True)
attribute2017 = merged1[merged1['TIME_PERIOD'] == 2017].sort_values('iso_o').reset_index(drop = True)
attribute2018 = merged1[merged1['TIME_PERIOD'] == 2018].sort_values('iso_o').reset_index(drop = True)
attribute2019 = merged1[merged1['TIME_PERIOD'] == 2019].sort_values('iso_o').reset_index(drop = True)
attribute2020 = merged1[merged1['TIME_PERIOD'] == 2020].sort_values('iso_o').reset_index(drop = True)

In [66]:
attribute2020.shape

(74, 10)

In [67]:
attribute2020.shape

(74, 10)

In [70]:
node_fix = pd.read_csv("../data/cleaned/nodal_fixed.csv")

In [73]:
node_fix.head(2)

,iso_o,country,landlocked,continent,lat,lon,langoff_1,colonizer1,category,development
0,ARG,Argentina,0,America,-34.666668,-58.500000,Spanish,ESP,Emerging and Developing Economies,0.0
1,AUT,Austria,1,Europe,48.216667,16.366667,German,NaN,Advanced Economies,1.0


In [72]:
at2000_fix = attribute2000.merge(node_fix, on = ['iso_o'], how = 'left')
at2001_fix = attribute2001.merge(node_fix, on = ['iso_o'], how = 'left')
at2002_fix = attribute2002.merge(node_fix, on = ['iso_o'], how = 'left')
at2003_fix = attribute2003.merge(node_fix, on = ['iso_o'], how = 'left')
at2004_fix = attribute2004.merge(node_fix, on = ['iso_o'], how = 'left')
at2005_fix = attribute2005.merge(node_fix, on = ['iso_o'], how = 'left')
at2006_fix = attribute2006.merge(node_fix, on = ['iso_o'], how = 'left')
at2007_fix = attribute2007.merge(node_fix, on = ['iso_o'], how = 'left')
at2008_fix = attribute2008.merge(node_fix, on = ['iso_o'], how = 'left')
at2009_fix = attribute2009.merge(node_fix, on = ['iso_o'], how = 'left')
at2010_fix = attribute2010.merge(node_fix, on = ['iso_o'], how = 'left')
at2011_fix = attribute2011.merge(node_fix, on = ['iso_o'], how = 'left')
at2012_fix = attribute2012.merge(node_fix, on = ['iso_o'], how = 'left')
at2013_fix = attribute2013.merge(node_fix, on = ['iso_o'], how = 'left')
at2014_fix = attribute2014.merge(node_fix, on = ['iso_o'], how = 'left')
at2015_fix = attribute2015.merge(node_fix, on = ['iso_o'], how = 'left')
at2016_fix = attribute2016.merge(node_fix, on = ['iso_o'], how = 'left')
at2017_fix = attribute2017.merge(node_fix, on = ['iso_o'], how = 'left')
at2018_fix = attribute2018.merge(node_fix, on = ['iso_o'], how = 'left')
at2019_fix = attribute2019.merge(node_fix, on = ['iso_o'], how = 'left')
at2020_fix = attribute2020.merge(node_fix, on = ['iso_o'], how = 'left')

In [74]:
at2020_fix.shape

(74, 19)

In [75]:
at2020_fix.head(2)

,iso_o,country_x,TIME_PERIOD,LTE,internet,gdp_pct,schooling,log_gdp_pct,D_log_gdp_pct,D_internet,country_y,landlocked,continent,lat,lon,langoff_1,colonizer1,category,development
0,ARG,Argentina,2020,1.0,85.500000,11393.050,18.36116,9.340758,-0.109103,5.599998,Argentina,0,America,-34.666668,-58.500,Spanish,ESP,Emerging and Developing Economies,0.0
1,AUS,Australia,2020,1.0,94.699997,58132.801,20.93944,10.970486,-0.013531,1.099999,Australia,0,Pacific,-35.299999,149.133,English,GBR,Advanced Economies,1.0


In [76]:
at2000_fix.to_csv("../data/cleaned/attribute2000.csv", encoding = 'utf-8', index = False)
at2001_fix.to_csv("../data/cleaned/attribute2001.csv", encoding = 'utf-8', index = False)
at2002_fix.to_csv("../data/cleaned/attribute2002.csv", encoding = 'utf-8', index = False)
at2003_fix.to_csv("../data/cleaned/attribute2003.csv", encoding = 'utf-8', index = False)
at2004_fix.to_csv("../data/cleaned/attribute2004.csv", encoding = 'utf-8', index = False)
at2005_fix.to_csv("../data/cleaned/attribute2005.csv", encoding = 'utf-8', index = False)
at2006_fix.to_csv("../data/cleaned/attribute2006.csv", encoding = 'utf-8', index = False)
at2007_fix.to_csv("../data/cleaned/attribute2007.csv", encoding = 'utf-8', index = False)
at2008_fix.to_csv("../data/cleaned/attribute2008.csv", encoding = 'utf-8', index = False)
at2009_fix.to_csv("../data/cleaned/attribute2009.csv", encoding = 'utf-8', index = False)
at2010_fix.to_csv("../data/cleaned/attribute2010.csv", encoding = 'utf-8', index = False)
at2011_fix.to_csv("../data/cleaned/attribute2011.csv", encoding = 'utf-8', index = False)
at2012_fix.to_csv("../data/cleaned/attribute2012.csv", encoding = 'utf-8', index = False)
at2013_fix.to_csv("../data/cleaned/attribute2013.csv", encoding = 'utf-8', index = False)
at2014_fix.to_csv("../data/cleaned/attribute2014.csv", encoding = 'utf-8', index = False)
at2015_fix.to_csv("../data/cleaned/attribute2015.csv", encoding = 'utf-8', index = False)
at2016_fix.to_csv("../data/cleaned/attribute2016.csv", encoding = 'utf-8', index = False)
at2017_fix.to_csv("../data/cleaned/attribute2017.csv", encoding = 'utf-8', index = False)
at2018_fix.to_csv("../data/cleaned/attribute2018.csv", encoding = 'utf-8', index = False)
at2019_fix.to_csv("../data/cleaned/attribute2019.csv", encoding = 'utf-8', index = False)
at2020_fix.to_csv("../data/cleaned/attribute2020.csv", encoding = 'utf-8', index = False)